In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re

%matplotlib inline

In [2]:
act = pd.read_csv('data/atusact_2014.dat')
cps = pd.read_csv('data/atuscps_2014.dat')
resp = pd.read_csv('data/atusresp_2014.dat')
rost = pd.read_csv('data/atusrost_2014.dat')
atussum = pd.read_csv('data/atussum_2014.dat') 
who = pd.read_csv('data/atuswho_2014.dat')

In [4]:
act.head()


,TUCASEID,TUACTIVITY_N,TEWHERE,TRTCCTOT_LN,TRTCC_LN,TRTCOC_LN,TRTEC_LN,TRTHH_LN,TRTNOHH_LN,TRTOHH_LN,...,TUCUMDUR24,TUEC24,TUSTARTTIM,TUSTOPTIME,TUTIER1CODE,TUTIER2CODE,TUTIER3CODE,TRCODE,TRTIER2,TXWHERE
0,20140101140007,1,-1,0,0,0,-1,0,-1,0,...,540,-1,04:00:00,13:00:00,1,1,1,10101,101,0
1,20140101140007,2,1,0,0,0,-1,0,-1,0,...,570,-1,13:00:00,13:30:00,3,1,1,30101,301,0
2,20140101140007,3,-1,60,60,0,-1,60,-1,60,...,630,-1,13:30:00,14:30:00,1,2,1,10201,102,0
3,20140101140007,4,13,10,10,0,-1,10,-1,10,...,640,-1,14:30:00,14:40:00,18,11,1,181101,1811,0
4,20140101140007,5,4,30,30,0,-1,30,-1,30,...,670,-1,14:40:00,15:10:00,11,1,1,110101,1101,0


In [31]:
cps.head()


KeyError: 'TRTHHFAMILY'

In [39]:
resp.head()
#resp['TRTHHFAMILY'] 
#resp['TRTNOCHILD']
new_data = pd.concat([resp['TRTHHFAMILY'], resp['TRTNOCHILD']], axis=1, keys=['1', '2'])
new_data.corr()

,1,2
1,1.000000,0.168112
2,0.168112,1.000000


In [7]:
rost.head()

,TUCASEID,TULINENO,TEAGE,TERRP,TESEX,TXAGE,TXRRP,TXSEX
0,20140101140007,1,20,19,2,0,0,0
1,20140101140007,2,19,21,1,0,0,0
2,20140101140007,3,0,22,1,0,0,0
3,20140101140011,1,16,18,2,0,0,0
4,20140101140011,2,33,24,2,0,0,0


In [30]:
atussum.rename(columns={'TEAGE':'Age', 'TESEX':'Sex', 'tucaseid':'ID', 't120302':'drug_use','t120301':'relaxing_thinking'}, inplace=True)
atussum.head()


,ID,TUFINLWGT,TRYHHCHILD,Age,Sex,PEEDUCA,PTDTRACE,PEHSPNON,GTMETSTA,TELFS,...,t181699,t181801,t181899,t189999,t500101,t500103,t500105,t500106,t500107,age
0,20140101140007,4891315.826760,0,20,2,39,1,1,1,4,...,0,0,0,0,0,0,0,0,0,20
1,20140101140011,23147368.359655,16,16,2,37,2,2,1,1,...,0,0,0,0,0,0,0,0,0,16
2,20140101140028,20941823.429044,2,65,2,38,1,1,1,5,...,0,0,0,0,0,0,0,0,0,65
3,20140101140063,2505899.417166,-1,50,2,39,2,2,2,5,...,0,0,0,0,0,0,0,0,0,50
4,20140101140168,15928231.823165,8,23,2,39,2,2,2,4,...,0,0,0,0,0,0,0,0,0,23


In [9]:
who.head()

,TUCASEID,TUACTIVITY_N,TRWHONA,TULINENO,TUWHO_CODE
0,20140101140007,1,1,-1,-1
1,20140101140007,2,0,3,22
2,20140101140007,3,1,-1,-1
3,20140101140007,4,0,2,21
4,20140101140007,4,0,3,22
